# import

In [ ]:
!git clone https://github.com/H999/GeneticCNN-torch.git
%cd GeneticCNN-torch/geneticCNN/

In [ ]:
import torch
import random
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

from lib.save_Individual import save_Individual
from lib.load_Individual import load_Individual
from lib.train_Individual import train_Individual

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('data', train=True, download=True, transform=transform)
testset = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)


def length(x): return len(x) * x.batch_size


print("Total train examples: ", length(train_loader))
print("Total test examples: ", length(test_loader))

In [ ]:
def set_seed(seed):
    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True


seed = 1234
set_seed(seed)

# class Stage

In [ ]:
from geneticCNN.Stage import Stage

# class Stages

In [ ]:
from geneticCNN.Stages import Stages

# class Individual

In [ ]:
from geneticCNN.Individual import Individual

# class Population

In [ ]:
from geneticCNN.Population import Population

# Train & test model for Individual

In [ ]:
EPOCHS = 5

In [ ]:
a = Individual((3, 5))
optimizer = torch.optim.SGD(a.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
train_Individual.train_Individual(a, device, train_loader, test_loader, optimizer, F.nll_loss, scheduler, EPOCHS, True)

In [ ]:
save_Individual.save_Individual(a, optimizer, scheduler)

In [ ]:
model, model_optimizer, model_scheduler = load_Individual.load_Individual()

In [ ]:
# model.state_dict()

In [ ]:
train_Individual.train_Individual(model, device, train_loader, test_loader, model_optimizer, F.nll_loss, model_scheduler, EPOCHS, True)

In [ ]:
import copy
b = copy.deepcopy(a)
optimizer = torch.optim.SGD(b.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
train_Individual.train_Individual(b, device, train_loader, test_loader, optimizer, F.nll_loss, scheduler, EPOCHS, True)

# Most similar with MNIST Experiments in paper 

In [ ]:
epochs = (20, 4, 1)
a = Population(train_loader, test_loader,
               torch.optim.Adam, {}, F.cross_entropy,
               torch.optim.lr_scheduler.MultiStepLR, {'milestones': [sum(epochs[0:i+1]) for i in range(len(epochs)-1)], 'gamma': 0.1}, sum(epochs), 20, crossover_rate=0.3, mutation_rate=0.1)
a.run(50)

# test

In [ ]:
# a=Stage(5,'1234567890')
# a=Stage(5,'1101100101')
# a=Stage(5,'1010000000')
# a=Stage(5,'1100000001')
# a=Stage(5,'0010001111')
# a=Stage(5,'0010001110')
# a=Stage(5)
# a=Stage(3)
# a=Stages(5)
# a=Stages(3)
# a=Individual(2)
a = Population(train_loader, test_loader,
               torch.optim.SGD, {'lr': 0.01, 'momentum': 0.9}, F.nll_loss,
               torch.optim.lr_scheduler.ExponentialLR, {'gamma': 0.95})
a.run()

In [ ]:
a = Population(train_loader, test_loader,
               torch.optim.Adam, {}, F.cross_entropy,
               torch.optim.lr_scheduler.ExponentialLR, {'gamma': 0.95})
a.run()

# test on tensorboard

In [ ]:
writer = SummaryWriter()
# writer.add_graph(a,images)

In [ ]:
# writer.close()

In [ ]:
# import time
# for i in range(1024):
#     print("{0:010b}".format(i))
#     try:
#         time.sleep(5)
#         writer = SummaryWriter()
#         writer.add_graph(Stage(5,"{0:010b}".format(i)),images)
#         writer.close()
#     except Exception as e:
#         print(e)

In [ ]:
!kill 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs